# Naïve substring similarity search

This notebook implements the naïve baseline algorithm: computing the [overlap coefficient](https://en.wikipedia.org/wiki/Overlap_coefficient) between the search query and all the plot summaries, and return the 10 highest scoring movies.

We implement this merely to compare the accuracy to the more sophisticated algorithms.

## Download and import packages

In [ ]:
!pip install -U strsimpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.3 MB/s 


In [ ]:
import ast
from strsimpy.overlap_coefficient import OverlapCoefficient
from tqdm.autonotebook import tqdm
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


## Setting hyperparameters and data preprocessing

In [ ]:
# Some hyperparameters

pre_prune_results = 100
results_to_show = 10

In [ ]:
# Mount drive and load datasets and model

from google.colab import drive
drive.mount("/content/gdrive")

plots = pd.read_csv("/content/gdrive/MyDrive/imdb_plots.csv", compression="zip", converters={'to_embed': ast.literal_eval})

plots['MovieId'] = plots.index
plots = plots.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

Mounted at /content/gdrive


In [ ]:
movie_ids = []
to_embed = []
for row in plots.iterrows():
  movie_id = row[1]['MovieId']
  for frag in row[1]['to_embed']:
    movie_ids.append(movie_id)
    to_embed.append(frag.lower())

id_and_summary = pd.DataFrame({'MovieId': movie_ids, 'to_embed': to_embed})

In [ ]:
movie_ids = []
queries = []
for row in plots.iterrows():
  movie_id = row[1]['MovieId']
  summ1 = row[1]['imdb_1']
  summ2 = row[1]['imdb_2']
  if not pd.isna(summ1):
    movie_ids.append(movie_id)
    queries.append(summ1.lower())
  if not pd.isna(summ2):
    movie_ids.append(movie_id)
    queries.append(summ2.lower())

test_queries = pd.DataFrame({'MovieId': movie_ids, 'summary': queries})

In [ ]:
oc = OverlapCoefficient()

## Defining the search function using the overlap coefficient

In [ ]:
def overlap_query(query_string, id_and_summary, wiki_dataset):
  query_string = query_string.lower()
  hits = []
  for row in id_and_summary.iterrows():
    movie_id = row[1]['MovieId']
    summary_fragment = row[1]['to_embed']
    score = oc.similarity(query_string, summary_fragment)
    hits.append((score, movie_id))
  hits.sort(key=lambda x: x[0], reverse=True)
  hits = hits[:pre_prune_results]

  results = []
  for raw_res in hits:
    if len(results) >= results_to_show:
      break
    
    score, movie_id = raw_res
    movie_title = wiki_dataset['Title'][movie_id]
    movie_year = wiki_dataset['Release Year'][movie_id]
    if movie_title.strip() not in map(lambda x: x[0][0].strip(), results):
      results.append(((movie_title, movie_year), score))
  return results

def measure_accuracy(query_dataset, id_and_summary, wiki_dataset):
  total = 0
  correct = 0

  for row in tqdm(query_dataset.iterrows()):
    query_string = row[1]['summary']
    movie_id = row[1]['MovieId']

    hits = overlap_query(query_string, id_and_summary, wiki_dataset)
    movie_title = wiki_dataset['Title'][movie_id]
    if movie_title.strip() in map(lambda x: x[0][0].strip(), hits):
      correct += 1
    total += 1

  return correct/total

## An example

Note that even after picking a substring of the plot summary of Shrek, none of the movies returned are the ones we were referencing.

In [ ]:
query = "His life is interrupted after the dwarfish"

overlap_query(query, id_and_summary, plots)

[(('Love and Death', 1975), 0.8421052631578947),
 (('Howard the Duck', 1986), 0.8157894736842105),
 (('Shallow Grave', 1994), 0.8157894736842105),
 (("Father's Little Dividend", 1951), 0.7894736842105263),
 (('Loving You', 1957), 0.7894736842105263),
 (("The Last Flight of Noah's Ark", 1980), 0.7894736842105263),
 (('Silent Night, Deadly Night', 1984), 0.7894736842105263),
 (('Class of 1999', 1990), 0.7894736842105263),
 (('Oscar', 1991), 0.7894736842105263),
 (('The Human Stain', 2003), 0.7894736842105263)]

## Testing performance on IMDB query set

Since this search function is rather slow, we only check the accuracy on 200 movies in the dataset.

In [ ]:
test_queries

,MovieId,summary
0,0,a chivalrous british officer takes the blame f...
1,0,"captain wynnegate leaves england, accepting th..."
2,1,a naive country girl is tricked into a sham ma...
3,1,"the callous rich, portrayed by lennox, think o..."
4,2,an extended family split up in france and germ...
...,...,...
9982,5268,four girls travel to a party in an isolated ho...
9983,5269,jae-hyuk is an ordinary man in his 40s. he wor...
9984,5270,esra working for a logistics firm lives with h...
9985,5271,recep ivedik has been depressed since the deat...


In [ ]:
test_queries_small = test_queries.head(200)

In [ ]:
measure_accuracy(test_queries_small, id_and_summary, plots)

0it [00:00, ?it/s]

0.21